In [ ]:
# Import libraries
import numpy as np 
from typing import Optional
import copy
from collections import defaultdict
from collections import deque

## Graphs

### Depth-first search

In [ ]:
# Example 1: 547. Number of Provinces

# There are n cities. 
# A province is a group of directly or indirectly connected cities and no other cities outside of the group. 
# You are given an n x n matrix isConnected 
# where isConnected[i][j] = isConnected[j][i] = 1 if the ithith city and the jthjth city are directly connected, 
# and isConnected[i][j] = 0 otherwise. 
# Return the total number of provinces.

class CityClusteringSolution:

    def collate_neighbours_dictionary(self, connections_array):
        neighbours_dictionary = defaultdict(list)

        for i in range(0, len(connections_array)):
            for j in range(1, len(connections_array[i])):
                if connections_array[i][j] == 1:
                    neighbours_dictionary[i].append(j)
                    neighbours_dictionary[j].append(i)

        return neighbours_dictionary


    def find_linked_cities(self, current_city, neighbours_dictionary, visited_set):
        visited_set_expanded = visited_set.copy()
        visited_set_expanded.add(current_city)
        current_neighbours = neighbours_dictionary[current_city]

        for neighbour in current_neighbours:
            if neighbour not in visited_set_expanded:        
                visited_set_expanded = self.find_linked_cities(neighbour, neighbours_dictionary, visited_set_expanded)

        return visited_set_expanded


    def count_number_of_city_clusters(self, isConnected):

        neighbours_dictionary = self.collate_neighbours_dictionary(isConnected)
        visited_set = set()
        number_of_clusters = 0

        for current_city in neighbours_dictionary.keys():
            if current_city not in visited_set:
                number_of_clusters += 1
                visited_set = self.find_linked_cities(current_city, neighbours_dictionary, visited_set)

        return number_of_clusters

isConnected1 = [
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
]

isConnected2 = [
    [0, 1, 0, 0],
    [1, 0, 0, 1],
    [0, 0, 0, 1],
    [0, 1, 1, 0]
]

print(CityClusteringSolution().count_number_of_city_clusters(isConnected1))
print(CityClusteringSolution().count_number_of_city_clusters(isConnected2))        


2
1


In [ ]:
# Example 2: 200. Number of Islands

# Given an m x n 2D binary grid which represents a map of 1 (land) and 0 (water),
# return the number of islands. 
# An island is surrounded by water and is formed by connecting adjacent land cells horizontally or vertically.

class CountIslandsSolution:

    def collate_land_bridges(self, grid):
        land_bridges = defaultdict(list)
        adjacent_directions = [
            (-1, 0),
            (0, -1),
            (+1, 0),
            (0, +1),
            ]

        for i in range(len(grid)):
            for j in range(len(grid[i])):
                if grid[i][j] == 1:
                    land_bridges[(i,j)] = []
                    for di, dj in adjacent_directions:
                        idash = i + di
                        jdash = j + dj
                        if 0 <= idash < len(grid) and 0 <= jdash < len(grid[i]) and grid[idash][jdash] == 1:
                            land_bridges[(i,j)].append((idash, jdash))
                    # in defaultdict, append() automatically creates a new key if one doesn't exist yet

        return land_bridges


    def join_contiguous_land_squares(self, land_square_coords, land_bridges, visited_land_squares):
        expanded_land_squares = visited_land_squares.copy()
        expanded_land_squares.add(land_square_coords)
        for neighbour in land_bridges[land_square_coords]:
            if neighbour not in visited_land_squares:
                expanded_land_squares = self.join_contiguous_land_squares(neighbour, land_bridges, expanded_land_squares)

        return expanded_land_squares


    def count_number_of_islands(self, grid):
        land_bridges = self.collate_land_bridges(grid)
        visited_land_squares = set()
        number_of_islands = 0

        for land_square in land_bridges.keys():
            if land_square not in visited_land_squares:
                number_of_islands += 1
                visited_land_squares |= self.join_contiguous_land_squares(land_square, land_bridges, visited_land_squares)

        return number_of_islands

grid1 = [
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 1]
]

grid2 = [
    [1, 0, 0, 0, 1],
    [1, 0, 1, 1, 1],
    [1, 0, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 1, 0, 1, 0]
]

print(CountIslandsSolution().count_number_of_islands(grid1))
print(CountIslandsSolution().count_number_of_islands(grid2))


3
4


In [ ]:
# Example 3: 1466. Reorder Routes to Make All Paths Lead to the City Zero

# There are n cities numbered from 0 to n - 1 and n - 1 roads 
# such that there is *only one way to travel between two different cities*. 
# Roads are represented by connections where connections[i] = [x, y] represents a road from city x to city y. 
# The edges are directed. 
# You need to swap the direction of some edges so that every city can reach city 0. 
# Return the minimum number of swaps needed.

def assemble_routes_set(edges):
    routes = set()
    for i, j in edges:
        routes.add((i, j))
    return routes

def assemble_neighbours_dictionary(edges):
    neighbours = defaultdict(list)
    for i, j in edges:
        neighbours[i].append(j)
        neighbours[j].append(i)
    return neighbours


def count_flips_in_tree(node, n_flips, nodes_seen, neighbours, routes):
    nodes_seen.add(node)

    if neighbours[node]:
        for i in range(len(neighbours[node])):
            current_neighbour = neighbours[node][i]
            if current_neighbour not in nodes_seen:
                nodes_seen.add(current_neighbour)
                if (current_neighbour, node) not in routes:
                    n_flips += 1 
                n_flips = count_flips_in_tree(current_neighbour, n_flips, nodes_seen, neighbours, routes)

    return n_flips


def calculate_flips_from_edges(edges):
    routes = assemble_routes_set(edges)
    neighbours = assemble_neighbours_dictionary(edges)

    root = 0
    nodes_seen = set()
    n_flips = 0

    total_flips = count_flips_in_tree(root, n_flips, nodes_seen, neighbours, routes)
    return total_flips


edges = [[0, 1], [2, 1], [3, 1]] 
print(calculate_flips_from_edges(edges))

# n1 = 5
# connections1 = [[1, 0], [1, 2], [3, 2], [3, 4]] 
# answer = 2; reverse connections [1, 2] and [3, 4]

# n2 = 4 
# connections2 = [[0, 1], [2, 1], [3, 1]] 
# # answer = 1


1


In [ ]:
# Example 4: 841. Keys and Rooms

# There are n rooms labeled from 0 to n - 1 
# and all the rooms are locked except for room 0. 
# Your goal is to visit all the rooms. 
# When you visit a room, you may find a set of distinct keys in it. 
# Each key has a number on it, denoting which room it unlocks, 
# and you can take all of them with you to unlock the other rooms. 
# Given an array rooms where rooms[i] is the set of keys that you can obtain if you visited room i, 
# return true if you can visit all the rooms, or false otherwise.

def assemble_keys_by_room(rooms):
    keys_by_room = defaultdict(list)

    for i in range(len(rooms)):
        keys_by_room[i] = rooms[i]

    return keys_by_room


def open_new_rooms(current_room, visited_rooms, keys_by_room):
    new_rooms = keys_by_room[current_room]
    for room in new_rooms:
        if room not in visited_rooms:
            visited_rooms.add(room)
            visited_rooms = open_new_rooms(room, visited_rooms, keys_by_room)

    return visited_rooms


def is_feasible_visit_all_rooms(rooms):
    initial_room = 0
    initial_visited_rooms = {initial_room}
    keys_by_room = assemble_keys_by_room(rooms)
    final_visited_rooms = open_new_rooms(initial_room, initial_visited_rooms, keys_by_room)
    all_rooms = set(keys_by_room.keys())
    if final_visited_rooms == all_rooms:
        return True
    else:
        return False
    

rooms1 = [[1], [2], [3], []]
print(is_feasible_visit_all_rooms(rooms1))

rooms2 = [[1], [2], [], [3]]
print(is_feasible_visit_all_rooms(rooms2))


defaultdict(<class 'list'>, {0: [1], 1: [2], 2: [3], 3: []})
True
False


### Breadth-first search

In [ ]:
# Example 1: 1091. Shortest Path in Binary Matrix

# Given an n x n binary matrix grid, 
# return the length of the shortest clear path in the matrix. 
# If there is no clear path, return -1. 
# A clear path is a path from the top-left cell (0, 0) to the bottom-right cell (n - 1, n - 1) 
# such that all visited cells are 0. 
# You may move 8-directionally (up, down, left, right, or diagonally).

def find_zero_neighbours(grid_matrix):
    directions = [
        [-1, -1],
        [-1, 0],
        [-1, +1],
        [0, -1],
        [0, +1],
        [+1, -1],
        [+1, 0],
        [+1, +1]
    ]

    neighbours_0 = defaultdict(list)
    ni = len(grid_matrix)
    nj = len(grid_matrix[0])
    for i in range(ni):
        for j in range(nj):
            for dx, dy in directions:
                xdash, ydash = i + dx, j + dy
                if 0 <= xdash < ni and 0 <= ydash < nj:
                    cell_value = grid_matrix[xdash][ydash]
                    if cell_value == 0:
                        neighbours_0[(i, j)].append(xdash, ydash)

    return neighbours_0

def find_shortest_path(grid_matrix):

    neighbours_0 = find_zero_neighbours(grid_matrix)
    ni = len(grid_matrix)
    nj = len(grid_matrix[0])
    starting_cell = (0, 0)

    if starting_cell == (ni - 1, nj - 1):
        return 0    # if 1x1 matrix, we're already at the goal

    target = (ni - 1, nj - 1)
    nodes_queue = deque([neighbours_0[starting_cell]]) # start at level 1
    seen_nodes = {starting_cell}
    current_level = 0

    while nodes_queue:
        current_level += 1
        current_level_length = len(nodes_queue)

        for _ in range(current_level_length):
            current_node = nodes_queue.popleft()
            seen_nodes.add(current_node)
            if current_node == target:
                return current_level
            else:
                neighbours = neighbours_0[current_node]
                if isinstance(neighbours, tuple): 
                    # prevents iterating over coordinates for a single tuple
                    if neighbours not in seen_nodes:
                        nodes_queue.append(neighbours)
                else:
                    for neighbour in neighbours:
                        if neighbour not in seen_nodes:
                            nodes_queue.append(neighbour)

    return -1


example_grid_matrix_1 = [
    [0]
    ]

example_grid_matrix_2 = [
    [0,1],
    [1,0]
    ]

example_grid_matrix_3 = [
    [0,1,1],
    [0,1,1],
    [0,0,0]
    ]

example_grid_matrix_4 = [
    [0,1,1],
    [0,1,1],
    [0,1,0]
    ]


print(find_shortest_path(example_grid_matrix_1))
print(find_shortest_path(example_grid_matrix_2))
print(find_shortest_path(example_grid_matrix_3))
print(find_shortest_path(example_grid_matrix_4))

# because we typically only visit a small fraction of nodes, it's more efficient to compute the neighbours on the fly

0
1
3
-1


In [ ]:
# Example 2: 863. All Nodes Distance K in Binary Tree

# Given the root of a binary tree, 
# a target node target in the tree, 
# and an integer k, 
# return an array of the values of all nodes 
# that have a distance k from the target node.

class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

# def add_parent_pointers_bfs(root: TreeNode):
#     node_queue = deque([root])
#     while node_queue:
#         level_length = len(node_queue)
#         for _ in range(level_length):
#             current_node = node_queue.popleft()
#             if current_node.left:
#                 current_node.left.parent = current_node
#                 node_queue.append(current_node.left)
#             if current_node.right:
#                 current_node.right.parent = current_node
#                 node_queue.append(current_node.right)
#     # Not used


def add_parents_to_tree(root):

    class TreeNodeWithParents:
        def __init__(self, val=0, left=None, right=None, parent=None):
            self.val = val
            self.left = left
            self.right = right
            self.parent = parent

    def add_parent_pointers_dfs(node: TreeNode, parent=None):
        if node is None:
            return None
        
        new_node = TreeNodeWithParents(val=node.val, parent=parent)
        new_node.left = add_parent_pointers_dfs(node.left, parent=new_node)
        new_node.right = add_parent_pointers_dfs(node.right, parent=new_node)
        
        return new_node

    tree_with_parents = add_parent_pointers_dfs(node=root)

    return tree_with_parents


def find_nodes_at_target_steps(target_node, target_steps):

    nodes_queue = deque([target_node])
    node_values = []
    seen_nodes = set()
    step_counter = 0

    while nodes_queue and step_counter <= target_steps:
        nodes_in_level = len(nodes_queue)
        for _ in range(nodes_in_level):
            current_node = nodes_queue.popleft()
            if current_node:
                if step_counter == target_steps:
                    node_values.append(current_node.val)
                else:
                    seen_nodes.add(current_node)
                    neighbours = [current_node.left, current_node.right, current_node.parent]
                    for neighbour in neighbours:
                        if neighbour not in seen_nodes:
                            nodes_queue.append(neighbour)
        step_counter += 1

    return node_values

# Example

root = TreeNode(val=5)
root.left = TreeNode(val=4, left=TreeNode(val=11, left=TreeNode(val=7), right=TreeNode(val=2)))
root.right = TreeNode(val=8, left=TreeNode(val=13), right=TreeNode(val=4, left=None, right=TreeNode(val=1)))

root_with_parents = add_parents_to_tree(root)
target_node = root_with_parents.right.left
target_steps = 2

print(find_nodes_at_target_steps(target_node, target_steps))

# Tree structure:
#         5
#        / \
#       4   8
#      /   / \
#     11  13  4
#    /  \      \
#   7    2      1

[4, 5]


In [ ]:
# Example 3: 542. 01 Matrix

# Given an m x n binary (every element is 0 or 1) matrix mat, 
# find the distance of the nearest 0 for each cell. 
# The distance between adjacent cells (horizontally or vertically) is 1.

def calculate_distance_of_all_cells_to_0(mat):

    nr = len(mat)
    nc = len(mat[0])

    dmat = mat # will be overwritten

    seen = set()
    cell_queue = deque()

    for r in range(nr):
        for c in range(nc):
            if mat[r][c] == 0:
                dmat[r][c] = 0
                seen.add((r,c))
                cell_queue.append((r,c))

    directions = [(0, +1), (0, -1), (-1, 0), (+1, 0)]  # Up, Right, Down, Left

    nsteps = 0
    while cell_queue:
        nsteps += 1
        length_current_level = len(cell_queue)
        for _ in range(length_current_level):
            r, c = cell_queue.popleft()
            for dr, dc in directions:
                rdash, cdash = r + dr, c + dc
                if 0 <= rdash < nr and 0 <= cdash < nc and (rdash, cdash) not in seen:
                    dmat[rdash][cdash] = nsteps
                    seen.add((rdash,cdash))
                    cell_queue.append((rdash,cdash))

    return dmat

mat = [[0,0,0],[0,1,0],[1,1,1]]
print(calculate_distance_of_all_cells_to_0(mat))

# return [[0,0,0],[0,1,0],[1,2,1]]

[[0, 0, 0], [0, 1, 0], [1, 2, 1]]


In [ ]:
# Example 4: 1293. Shortest Path in a Grid with Obstacles Elimination

# You are given an m x n integer matrix grid where each cell is either 0 (empty) or 1 (obstacle). 
# You can move up, down, left, or right from and to an empty cell in one step. 
# Return the minimum number of steps to walk from the upper left corner to the lower right corner 
# given that you can eliminate at most k obstacles. 
# If it is not possible, return -1.

def is_valid_cell(coords, coords_max):
    valid_row = 0 <= coords[0] <= coords_max[0]
    valid_col = 0 <= coords[1] <= coords_max[1]
    if valid_row and valid_col:
        return True
    else:
        return False

def find_shortest_path_with_removals(grid_matrix, k):
    
    directions = [(0, +1), (0, -1), (-1, 0), (+1, 0)] 

    row_start = 0
    col_start = 0
    n_steps_start = 0
    removals_start = k
    coords_max = (len(grid_matrix) - 1, len(grid_matrix[0]) - 1)

    seen_nodes = set()
    intial_state = (row_start, col_start, n_steps_start, removals_start)
    node_queue = deque([intial_state])

    while node_queue:

        nodes_in_current_level = len(node_queue)

        for _ in range(nodes_in_current_level):

            current_state = node_queue.popleft()
            row, col, nsteps, removals = current_state
            new_steps = nsteps + 1

            for drow, dcol in directions:

                rdash, cdash = row + drow, col + dcol
                coords = (rdash, cdash) 

                if coords == coords_max:
                    return new_steps

                if is_valid_cell(coords, coords_max):

                    cell_value = grid_matrix[rdash][cdash]
                    new_removals = removals
                    
                    if cell_value == 1:
                        new_removals -= 1

                    if new_removals >= 0:
                        new_state = (rdash, cdash, new_steps, new_removals)
                        if new_state not in seen_nodes:
                            seen_nodes.add(new_state)
                            node_queue.append(new_state)
                        


grid_matrix = [
    [0,1,1],
    [0,1,1],
    [1,1,0]
    ]

print(find_shortest_path_with_removals(grid_matrix, k=2))


4


In [ ]:
# Example 5: 1129. Shortest Path with Alternating Colors

# You are given a directed graph with n nodes labeled from 0 to n - 1. 
# Edges are red or blue in this graph. 
# You are given redEdges and blueEdges, 
# where redEdges[i] and blueEdges[i] both have the format [x, y] indicating an edge from x to y in the respective color. 
# Return an array ans of length n, 
# where answer[i] is the length of the shortest path from 0 to i where edge colors alternate, 
# or -1 if no path exists.

# All-red example:

redEdges = [
    (0, 1),
    (0, 2),
    (1, 2),
    (1, 3),
    (2, 3),
    (3, 4),
    (4, 0),
    (4, 5),
    (5, 6),
    (5, 7),
    (6, 7),
    (7, 8),
    (8, 5)  # This creates a cycle
]

def assemble_neighbours_dictionary(edges: list):
    neighbours = defaultdict(list)
    for start, end in edges:
        neighbours[start].append(end)
    return neighbours

# IGNORING NODE COLOUR

def find_shortest_paths_for_all_nodes(edges):

    neighbours = assemble_neighbours_dictionary(edges)

    start_node = 0
    current_level = 0
    seen_nodes = set()
    shortest_path = [-1] * (max(neighbours.keys()) + 1)
    node_queue = deque([start_node])

    while node_queue:
        nodes_in_level = len(node_queue)
        for _ in range(nodes_in_level):
            current_node = node_queue.popleft()
            if current_node not in seen_nodes:
                seen_nodes.add(current_node)
                shortest_path[current_node] = current_level
                for neighbour in neighbours[current_node]:
                    if neighbour not in seen_nodes:
                        node_queue.append(neighbour)
        current_level += 1

    return(shortest_path)

print(find_shortest_paths_for_all_nodes(redEdges))

# ACCOUNTING FOR NODE COLOUR

redEdges = [
    (0, 1),
    (1, 2),
    (2, 3),
    (4, 0),
    (5, 6),
    (6, 7),
    (8, 5)  # This creates a cycle
]

blueEdges = [
    (0, 2),
    (1, 3),
    (3, 4),
    (4, 5),
    (5, 7),
    (7, 8),
]

red_neighbours = assemble_neighbours_dictionary(redEdges)
blue_neighbours = assemble_neighbours_dictionary(blueEdges)

def find_shortest_paths_for_all_nodes_alternating(edges_1, edges_2):
    
    first_neighbours = assemble_neighbours_dictionary(edges_1)
    second_neighbours = assemble_neighbours_dictionary(edges_2)

    node_set = set(first_neighbours.keys()) | set(second_neighbours.keys())

    start_node = 0
    current_level = 0
    seen_nodes = set()
    shortest_path = [-1] * (max(node_set) + 1)
    node_queue = deque([start_node])

    while node_queue:
        nodes_in_level = len(node_queue)
        for _ in range(nodes_in_level):
            current_node = node_queue.popleft()
            if current_node not in seen_nodes:
                seen_nodes.add(current_node)
                shortest_path[current_node] = current_level
                if current_level % 2 == 0:
                    valid_neighbours = first_neighbours[current_node]
                else: 
                    valid_neighbours = second_neighbours[current_node]
                for neighbour in valid_neighbours:
                    if neighbour not in seen_nodes:
                        node_queue.append(neighbour)
        current_level += 1

    return(shortest_path)

def find_shortest_paths_for_all_nodes_alternating_either_route(edges_1, edges_2):
    one = find_shortest_paths_for_all_nodes_alternating(edges_1=edges_1, edges_2=edges_2)
    two = find_shortest_paths_for_all_nodes_alternating(edges_1=edges_2, edges_2=edges_1)

    shortest_paths = []

    for i in range(len(one)):
        if one[i] != -1 or two[i] != -1:
            shortest_paths.append(min(one[i], two[i]))
        else:
            shortest_paths.append(-1)

    return shortest_paths

print(find_shortest_paths_for_all_nodes_alternating_either_route(redEdges, blueEdges))


[0, 1, 1, 2, 3, 4, 5, 5, 6]
[0, -1, -1, 2, -1, -1, -1, -1, -1]
